In [93]:
# Import packages
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
from groq import Groq

In [94]:
# Header to set the requests as a browser requests
headers = {
    'authority': 'www.amazon.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-language': 'en-US,en;q=0.9,bn;q=0.8',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="102", "Google Chrome";v="102"',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
}

In [95]:
# URL of The amazon Review page
reviews_url = 'https://www.amazon.co.jp/-/en/Anti-fatigue-Ergonomic-Reclining-Resilience-Breathable/product-reviews/B0BML7RL3P/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews'

In [96]:
# Define Page No
len_page = 1

In [97]:
# Extra Data as Html object from amazon Review page
def reviewsHtml(url, len_page):
    
    # Empty List define to store all pages html data
    soups = []
    
    # Loop for gather all 3000 reviews from 300 pages via range
    for page_no in range(1, len_page + 1):
        
        # parameter set as page no to the requests body
        params = {
            'ie': 'UTF8',
            'reviewerType': 'all_reviews',
            'filterByStar': 'critical',
            'pageNumber': page_no,
        }
        
        # Request make for each page
        response = requests.get(url, headers=headers)
        
        # Save Html object by using BeautifulSoup4 and lxml parser
        soup = BeautifulSoup(response.text, 'lxml')
        
        # Add single Html page data in master soups list
        soups.append(soup)
        
    return soups

In [98]:
# Grab Reviews name, description, date, stars, title from HTML
def getReviews(html_data):

    # Create Empty list to Hold all data
    data_dicts = []
    
    # Select all Reviews BOX html using css selector
    boxes = html_data.select('div[data-hook="review"]')
    
    # Iterate all Reviews BOX 
    for box in boxes:
        
        # Select Name using css selector and cleaning text using strip()
        # If Value is empty define value with 'N/A' for all.
        try:
            name = box.select_one('[class="a-profile-name"]').text.strip()
        except Exception as e:
            name = 'N/A'

        try:
            stars = box.select_one('[data-hook="review-star-rating"]').text.strip().split(' out')[0]
        except Exception as e:
            stars = 'N/A'   

        try:
            title = box.select_one('[data-hook="review-title"]').text.strip()
        except Exception as e:
            title = 'N/A'

        try:
            # Convert date str to dd/mm/yyy format
            datetime_str = box.select_one('[data-hook="review-date"]').text.strip().split(' on ')[-1]
            date = datetime.strptime(datetime_str, '%B %d, %Y').strftime("%d/%m/%Y")
        except Exception as e:
            date = 'N/A'

        try:
            description = box.select_one('[data-hook="review-body"]').text.strip()
        except Exception as e:
            description = 'N/A'

        # create Dictionary with al review data 
        data_dict = {
            'Name' : name,
            'Stars' : stars,
            'Title' : title,
            'Date' : date,
            'Description' : description
        }

        # Add Dictionary in master empty List
        data_dicts.append(data_dict)
    
    return data_dicts

In [99]:
# Grab all HTML
html_datas = reviewsHtml(reviews_url, len_page)

In [100]:
# Empty List to Hold all reviews data
reviews = []

In [101]:
# Iterate all Html page 
for html_data in html_datas:
    
    # Grab review data
    review = getReviews(html_data)
    
    # add review data in reviews empty list
    reviews += review

In [102]:
# Create a dataframe with reviews Data
df_reviews = pd.DataFrame(reviews)

In [103]:
df_reviews

,Name,Stars,Title,Date,Description
0,ギヤさん,5.0,5.0 out of 5 stars\n期待以上の椅子,28/03/2024,座り心地も良く機能性に加えてデザインも気に入っています。組み立ても動画を見ながら進めましたの...
1,しょう,1.0,1.0 out of 5 stars\n絶対に買わないで,19/04/2024,不良品です。組み立て式ですが、ネジ穴の位置が大幅にずれていていました。酷すぎて星1すら付けた...
2,ごま,2.0,2.0 out of 5 stars\n説明書が酷すぎる,27/04/2024,説明書がいい加減すぎて組むのが大変でした。ネジのサイズと個数が合っていない、付けるべきパーツ...
3,クンパンチ,3.0,3.0 out of 5 stars\n部品に問題あり,14/04/2024,寸法の違うネジが入っていた。恐らく、違う製品の部品を入れたのだろう。コスパがいいと、よくある...
4,ちょろぎ,4.0,4.0 out of 5 stars\n一部組みにくい,19/04/2024,アームレストのビス穴が１ヶ所、若干ずれがあるので組み立てにやや難がある。ただしちょっと頑張れ...
5,きんちゃん,5.0,5.0 out of 5 stars\n使用良好,03/08/2023,・非常に良コスパです・座面のクッション性も丁度良く座り心地も良いです・座面広さも実測奥行46...
6,Amazon カスタマー,2.0,2.0 out of 5 stars\nこんなもんかな,10/04/2024,お値段通り、細かい事が気になる人はやめたほうが良いです。良い物は、それなりの値段ですよね。長...
7,アマゾン太郎,3.0,3.0 out of 5 stars\n値段の割にはよいと思うが・・・,30/03/2024,①組み立て説明が分かりにくい。ビスの種類に混乱した。②アームレストの建付けが悪く、シートに固...
8,yui,4.0,4.0 out of 5 stars\n値段相応オフィスチェアー,20/04/2024,メッシュの背もたれ、肘置きが跳ね上げできる。
9,金志泰,5.0,5.0 out of 5 stars\n対応いい！,02/03/2023,こちらの間違いで組み立て間違えたんですが、それでも親切に対応してくれました！椅子もすごくいいです！


In [87]:
# Save data
#df_reviews.to_csv('reviews.csv', index=False)

In [104]:
descriptions = df_reviews['Description']

In [109]:
import os

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": f"Analyzing the {descriptions} give me the general opinion of the product and suggestions for how to imporve, increase the sales and get better reviews",
        }
    ],
    model="llama3-70b-8192",
)

print(chat_completion.choices[0].message.content)

Based on the reviews, here's a general opinion of the product and suggestions for improvement:

**General Opinion:**
The product has received mixed reviews. Some customers are satisfied with the product's design, functionality, and value for money. However, others have experienced issues with assembly, poor instruction manuals, and defects in the product.

**Positive Aspects:**

* Comfortable seating and good design
* Good value for money
*Mesh backrest and adjustable armrest are appreciated features

**Negative Aspects:**

* Difficulty in assembly due to poor instruction manuals and mismatched parts
* Defects in the product, such as incorrectly sized screws or incorrect parts
* Some customers find the product not worth the price due to quality issues

**Suggestions for Improvement:**

1. **Improve the instruction manual:** Make the assembly instructions clearer and more detailed to reduce frustration and errors during assembly.
2. **Quality Control:** Implement stricter quality contro